In [30]:
using Plots
using Printf

In [46]:
f(x, y) = 3x^2 + 2y^2
h(x, y) = 1 - x - y
dL(x, c) = [6x[1] - c / (x[1] + x[2] - 1), 4x[2] - c / (x[1] + x[2] - 1)]

dL (generic function with 1 method)

In [47]:
function BFGS(x, c, epsilon)
    Hinv = [1 0; 0 1]
    xold = copy(x)
    xnew = xold - 0.01 .* Hinv * dL(xold, c)
    while sum((xnew - xold).^2) > epsilon
        s = xnew - xold
        t = dL(xnew, c) - dL(xold, c)
        xold = copy(xnew)
        Hinv = (
            Hinv + (s' * t + t' * Hinv * t) * s * s' ./ (s' * t)^2 -
            (Hinv * t * s' + s * t' * Hinv) ./ (s' * t)
        )
        xnew = xold - 0.01 .* Hinv * dL(xold, c)
    end
    xnew
end

BFGS (generic function with 2 methods)

## BFGSの正当性チェック

In [49]:
df(x) = [10x[1], 2x[2]]
pos = 10 .* rand(2) .- 5

2-element Array{Float64,1}:
 -3.688358824660607 
  1.7337900266387773

In [50]:
function BFGS(x, epsilon)
    Hinv = [1 0; 0 1]
    xold = copy(x)
    xnew = xold - Hinv * df(x)
    while sqrt(sum((xnew - xold).^2)) > epsilon
        s = xnew - xold
        t = df(xnew) - df(xold)
        xold = copy(xnew)
        Hinv = (
            Hinv + (s' * t + t' * Hinv * t) * s * s' ./ (s' * t)^2 -
            (Hinv * t * s' + s * t' * Hinv) ./ (s' * t)
        )
        xnew = xold - Hinv * df(xold)
    end
    xnew
end

BFGS (generic function with 3 methods)

In [25]:
BFGS(pos, 1e-6)

2-element Array{Float64,1}:
  1.1287416632089007e-12
 -3.4080913071973092e-15

## Barrier Method

In [51]:
function barrier(x, eps_barrier, eps_bfgs)
    xtrail = []
    ytrail = []
    append!(xtrail, x[1])
    append!(ytrail, x[2])

    xold = copy(x)
    xnew = BFGS(xold, 100.0, eps_bfgs)
    append!(xtrail, xnew[1])
    append!(ytrail, xnew[2])
    c = 1.0
    while sum((xnew - xold).^2) > eps_barrier
        c += 1.0
        xold = copy(xnew)
        xnew = BFGS(xold, 100. / c, eps_bfgs)
        append!(xtrail, xnew[1])
        append!(ytrail, xnew[2])
    end
    xtrail, ytrail
end

barrier (generic function with 1 method)

In [52]:
eps_barrier = 1e-10
eps_bfgs = 1e-16
x = [1., 1.]

2-element Array{Float64,1}:
 1.0
 1.0

In [53]:
xtrail, ytrail = barrier(x, eps_barrier, eps_bfgs)

(Any[1.0, 2.78972, 2.03666, 1.70407, 1.5064, 1.37189, 1.2729, 1.19618, 1.13452, 1.0836  …  0.409657, 0.409652, 0.409646, 0.40964, 0.409635, 0.409629, 0.409624, 0.409618, 0.409613, 0.409607], Any[1.0, 4.18458, 3.055, 2.5561, 2.25959, 2.05784, 1.90935, 1.79428, 1.70179, 1.62539  …  0.614486, 0.614478, 0.61447, 0.614461, 0.614453, 0.614445, 0.614436, 0.614428, 0.61442, 0.614411])

In [212]:
x = y = range(-4.0, 4.0, length=1000)
p = plot(x, y, f, st=[:contourf])
for i in 1:length(xtrail)-1
    plot!(p[1], [xtrail[i], xtrail[i+1]], [ytrail[i], ytrail[i+1]], line=(:white, 1), legend=:none)
end
posx = @sprintf "%.2f" xtrail[end]
posy = @sprintf "%.2f" ytrail[end]
annotate!(xtrail[end], ytrail[end], text("($(posx), $(posy))", 10, :center, :white))
png("barrier")

In [208]:
@sprintf "%.2f" ytrail[end]

"0.61"

## Penalty Method

In [11]:
function BFGS(x, epsilon, f, c...)
    Hinv = [1 0; 0 1]
    xold = copy(x)
    df = ifelse(length(c) == 0, f(xold, 0), f(xold, c[1]))
    xnew = xold - 0.01 .* Hinv * df
    while sum((xnew - xold).^2) > epsilon
        s = xnew - xold
        t = ifelse(length(c) == 0, f(xnew, 0) - f(xold, 0), f(xnew, c[1]) - f(xold, c[1]))
        xold = copy(xnew)
        Hinv = (
            Hinv + (s' * t + t' * Hinv * t) * s * s' ./ (s' * t)^2 -
            (Hinv * t * s' + s * t' * Hinv) ./ (s' * t)
        )
        df = ifelse(length(c) == 0, f(xold, 0), f(xold, c[1]))
        xnew = xold - 0.01 .* Hinv * df
    end
    xnew
end

BFGS (generic function with 1 method)

In [12]:
h(x, y) = 1 - x - y
nablaL(x, c) = [
    6x[1] + c * ifelse(max(h(x[1], x[2]), 0) == 0, 0, -2 * h(x[1], x[2])),
    4x[2] + c * ifelse(max(h(x[1], x[2]), 0) == 0, 0, -2 * h(x[1], x[2]))
]

nablaL (generic function with 1 method)

In [35]:
function penalty(x, eps_penalty, eps_bfgs)
    xtrail = []
    ytrail = []
    append!(xtrail, x[1])
    append!(ytrail, x[2])

    xold = copy(x)
    c = 1
    xnew = BFGS(xold, eps_bfgs, nablaL, c)
    append!(xtrail, xnew[1])
    append!(ytrail, xnew[2])
    while sum((xnew - xold).^2) > eps_penalty
        c *= 1.01
        xold = copy(xnew)
        xnew = BFGS(xold, eps_bfgs, nablaL, c)
        append!(xtrail, xnew[1])
        append!(ytrail, xnew[2])
    end
    xtrail, ytrail
end

penalty (generic function with 1 method)

In [42]:
eps_penalty = 1e-9
eps_bfgs = 1e-16
x = [1., 1.]

2-element Array{Float64,1}:
 1.0
 1.0

In [43]:
xtrail, ytrail = penalty(x, eps_penalty, eps_bfgs)

(Any[1.0, 0.181819, 0.182805, 0.183793, 0.184782, 0.185772, 0.186762, 0.187753, 0.188744, 0.189737  …  0.398092, 0.398111, 0.398129, 0.398148, 0.398166, 0.398184, 0.398202, 0.39822, 0.398237, 0.398255], Any[1.0, 0.272728, 0.274207, 0.275689, 0.277173, 0.278657, 0.280143, 0.281629, 0.283117, 0.284605  …  0.597137, 0.597165, 0.597193, 0.597221, 0.597248, 0.597276, 0.597302, 0.597329, 0.597355, 0.597381])

In [44]:
f(x, y) = 3x^2 + 2y^2

f (generic function with 1 method)

In [45]:
x = y = range(-1.0, 1.0, length=1000)
p = plot(x, y, f, st=[:contourf])
for i in 1:length(xtrail)-1
    plot!(p[1], [xtrail[i], xtrail[i+1]], [ytrail[i], ytrail[i+1]], line=(:white, 1), legend=:none)
end
posx = @sprintf "%.2f" xtrail[end]
posy = @sprintf "%.2f" ytrail[end]
annotate!(xtrail[end], ytrail[end], text("($(posx), $(posy))", 10, :center, :white))
png("penalty")